Dependencies

docx: pip install python-docx

PyPDF2: pip install pypdf2

NLTK  : pip install -U nltk

XLRD pip install xlrd

In [133]:
import glob2
import os
import docx
import xlrd
from docx import Document
import PyPDF2
import csv
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [39]:
#Check if have download all needed additional data for nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('movie_reviews')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxpr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\maxpr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\maxpr/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\maxpr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\maxpr/nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

### Openning files

In [6]:
keppels_files = glob2.glob('files/Keppel/*')
prudential_files = glob2.glob('files/Prudential/*')
print(f'we have {len(keppels_files)} keppels documents and {len(prudential_files)} Prudential documents')

we have 218 keppels documents and 215 Prudential documents


Let's check the percentage of different document type

In [7]:
keppels_files_pdf = glob2.glob('files/Keppel/*.pdf')
keppels_files_xls = glob2.glob('files/Keppel/*.xlsx') + glob2.glob('files/Keppel/*.xls')
keppels_files_doc = glob2.glob('files/Keppel/*.docx') + glob2.glob('files/Keppel/*.doc')
len(keppels_files_pdf) + len(keppels_files_xls) + len(keppels_files_doc)

193

In [8]:
prudential_files_pdf = glob2.glob('files/Prudential/*.pdf')
prudential_files_xls = glob2.glob('files/Prudential/*.xlsx') + glob2.glob('files/Prudential/*.xls')
prudential_files_doc = glob2.glob('files/Prudential/*.docx') + glob2.glob('files/Prudential/*.doc')
len(prudential_files_pdf) + len(prudential_files_xls) + len(prudential_files_doc)

194

From that we can conclude that the documents are of three different types (pdf, word and excel)

In [9]:
doc = Document(keppels_files_doc[0])
def getText(filename):
    if(filename.endswith("docx")):
        doc = docx.Document(filename)
    else:
        doc = doc.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [10]:
print(f'File {keppels_files_doc[3]} content is :')
print((getText(keppels_files_doc[3])))

File files/Keppel\CYP%20Meeting%20May%202017docx.docx content is :
CHILDREN AND YOUNG PEOPLE NETWORK
18.05.2017 at London Funders

Participants 
         Deborah	Meyer	Big Lottery Fund 
	Toyin	Ogundana	CAF Venturesome
	Daisy	Ryan	Henry Smith Charity
	Erik	Mesel	John Lyon's Charity
	Sharon	Long	Partnership For Young London
	Mihai	Calin Bica	Roma Support Group
	Gaba	Smolinska-	Roma Support Group
	Poffley
	Victoria	Cousins	Safer London Foundation 
	Cara	Whelan	Save the Children
	Emma	Hale	Sir John Cass's Foundation
	Rachel	Earnshaw	The Big Give
	Ciaran	Rafferty	The City Bridge Trust
	Jenny	Oppenheimer	The Pilgrim Trust
	Patrick	Kinsella	Traveller Movement
	Yvonne	MacNamara	Traveller Movement
	Flora	Ward	Youth Music

In attendance
         Geraldine	Tovey	London Funders
	David	Warner	London Funders

Welcome and apologies for absence

Ciaran welcomed everyone to the group and introductions were made. He welcomed the group to its first network meeting in 2017 following the Vision for Young L

In [11]:
#keep : no not don't aren't aren
ourStopWord = ['i', 'me', 'my', 'myself', 'we', 'our','I',
               'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
               'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
               'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',
               'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
               'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do',
               'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
               'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
               'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
               'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
               'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',
                'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will',
               'just','should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
                'ma' ]


In [12]:
def removeStop(text):
    stop_words = ourStopWord
 
    word_tokens = word_tokenize(text)
 
    #filtered_sentence = [w for w in word_tokens if not w in stop_words]
 
    filtered_sentence = []
 
    for w in word_tokens:
        #w = w.lower()
        if w not in stop_words:
            if (w=="n\'t"):
                w="not"
            filtered_sentence.append(w)
    return filtered_sentence

In [13]:
text = removeStop(getText(keppels_files_doc[3]))

In [14]:
def textPOS(textToPOS):
    #text = [c.lower() for c in text]
    return nltk.pos_tag(textToPOS)

In [15]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [16]:
textCleaned = removeStop(getText(keppels_files_doc[3]))
print(textCleaned)

['CHILDREN', 'AND', 'YOUNG', 'PEOPLE', 'NETWORK', '18.05.2017', 'London', 'Funders', 'Participants', 'Deborah', 'Meyer', 'Big', 'Lottery', 'Fund', 'Toyin', 'Ogundana', 'CAF', 'Venturesome', 'Daisy', 'Ryan', 'Henry', 'Smith', 'Charity', 'Erik', 'Mesel', 'John', 'Lyon', "'s", 'Charity', 'Sharon', 'Long', 'Partnership', 'For', 'Young', 'London', 'Mihai', 'Calin', 'Bica', 'Roma', 'Support', 'Group', 'Gaba', 'Smolinska-', 'Roma', 'Support', 'Group', 'Poffley', 'Victoria', 'Cousins', 'Safer', 'London', 'Foundation', 'Cara', 'Whelan', 'Save', 'Children', 'Emma', 'Hale', 'Sir', 'John', 'Cass', "'s", 'Foundation', 'Rachel', 'Earnshaw', 'The', 'Big', 'Give', 'Ciaran', 'Rafferty', 'The', 'City', 'Bridge', 'Trust', 'Jenny', 'Oppenheimer', 'The', 'Pilgrim', 'Trust', 'Patrick', 'Kinsella', 'Traveller', 'Movement', 'Yvonne', 'MacNamara', 'Traveller', 'Movement', 'Flora', 'Ward', 'Youth', 'Music', 'In', 'attendance', 'Geraldine', 'Tovey', 'London', 'Funders', 'David', 'Warner', 'London', 'Funders', 'W

In [17]:
#textPOS = textPOS(textCleaned)

In [64]:
def removeUnknown(text):
    res = []
    for tup in text:
        print(t)

In [19]:
def lemmize(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    finalStemmed = []
    for w in text : 
        allInfo = w
        word = allInfo[0]
        tag = allInfo[1]
        stemmedWord = ""
        try:
            stemmedWord = wordnet_lemmatizer.lemmatize(word,get_wordnet_pos(tag))
            finalStemmed.append((stemmedWord,w[1][0]))
        except Exception:
            finalStemmed.append((w[0],w[1][0]))     
    return finalStemmed

In [20]:
print(lemmize(textPOS[:100]))

TypeError: 'function' object is not subscriptable

In [ ]:
print(keppels_files_pdf[84])
pdf = PyPDF2.PdfFileReader(keppels_files_pdf[4],'rb')
pdf.getNumPages()
print(pdf.getPage(6).extractText())

In [ ]:
pdf = PyPDF2.PdfFileReader("files/Keppel\\2017-DeepwaterPoster-D6-Ads.pdf",'rb')
print(pdf.getPage(0).extractText())

In [ ]:
def extractTextPDF(pdf):
    try:
        doc = PyPDF2.PdfFileReader(pdf,'r')
    except:
        return None
    return result

In [ ]:
def countPDF(pdf,comp):
    count = 0
    result = []
    count = 0
    doc = extractText(pdf)
    if(doc is not None):
        index = doc.getNumPages()
        #print(index)
        if(index<100):
            for i in range(0,index):
                txt = doc.getPage(i).extractText()
                allS = txt.split(' ')
                for s in allS:
                    if(comp in s.lower()):
                        count+=1
    else:
        print("COULD NOT OPEN THE PDF ", pdf)
    return count

In [ ]:
def extractDoc(text):
    txt = text[:-3]+'txt'
    f = open(txt)
    data = f.read()
    return data

In [ ]:
def countDoc(text,comp):
    count = 0
    txt = extractDoc(text)
    if (txt is not None):
        allS = txt.split(' ')
        for s in allS:
            if(comp in s.lower()):
                    count+=1
    else:
        print("COULD NOT OPEN DOC(X)", text )
    return count

In [214]:
def extractEx(path):
    strRe = ""
    data = xlrd.open_workbook(path)
    sheet = data.sheet_by_index(0)
    result = []
    for rownum in range(sheet.nrows): # sh1.nrows -> number of rows (ncols -> num columns)
        result.append((sheet.row_values(rownum)))
        print(result[:5])
    #print(result)
    for list1 in result:
        #print(list1)
        strRe += ''.join(str(e) for e in list(list1))
    return strRe

In [215]:
print(extractEx("files/Keppel/Adore+Home+Magazine+2_Stockists+List.xlsx"))

[['Name', 'Address 1', 'Address 2', 'Suburb', 'State', 'Postcode', 'Dist']]
[['Name', 'Address 1', 'Address 2', 'Suburb', 'State', 'Postcode', 'Dist'], ['ABBOTSFORD POINT NEWSAGENCY', 'TANG C, & FENG, L', '545 GREAT NORTH ROAD', 'ABBOTSFORD', 'NSW', '2046', 2.0]]
[['Name', 'Address 1', 'Address 2', 'Suburb', 'State', 'Postcode', 'Dist'], ['ABBOTSFORD POINT NEWSAGENCY', 'TANG C, & FENG, L', '545 GREAT NORTH ROAD', 'ABBOTSFORD', 'NSW', '2046', 2.0], ['MAGNA GROUP PTY LTD', 'MAGNA GROUP PTY LTD', 'SHOP 51/52 HUB DRIVE', 'ABERFOYLE PARK', 'SA', '5159', 3.0]]
[['Name', 'Address 1', 'Address 2', 'Suburb', 'State', 'Postcode', 'Dist'], ['ABBOTSFORD POINT NEWSAGENCY', 'TANG C, & FENG, L', '545 GREAT NORTH ROAD', 'ABBOTSFORD', 'NSW', '2046', 2.0], ['MAGNA GROUP PTY LTD', 'MAGNA GROUP PTY LTD', 'SHOP 51/52 HUB DRIVE', 'ABERFOYLE PARK', 'SA', '5159', 3.0], ['ADAMSTOWN NEWSAGENCY', 'SIWA-TECH PTY LTD T/AS', '259 BRUNKER ROAD', 'ADAMSTOWN', 'NSW', '2289', 2.0]]
[['Name', 'Address 1', 'Address 2', '

In [ ]:
def mapperOpener(path):
    mapping = []
    with open(path, 'r') as csvfile:
        spamreader = csv.reader(csvfile,delimiter='|')
        for row in spamreader:
            mapping.append((row[0],row[1],row[2]))
    return mapping

In [ ]:
filesMap = mapperOpener("submission_mapper.csv")

In [ ]:
keppel = 'files/Keppel\\'
prudential = 'files/Prudential\\'
## Checked all are included in the if clauses
def fromMapToOpener(fileMap):
    for line in fileMap:
        count = 0
        ID = line[0]
        fileName = line[1]
        compName = line[2]
        ##pdf
        if(fileName.endswith('pdf')):
            if(compName == 'Keppel'):
                count = countPDF(keppel+fileName,'keppel')
            else:
                count = countPDF(prudential+fileName,'prudential')
        ##doc
        elif((fileName.endswith('doc') or fileName.endswith('docx'))):
            if(compName == 'Keppel'):
                count = countDoc(keppel+fileName,'keppel')
            else:
                count = countDoc(prudential+fileName,'prudential')
        ##excel
        elif(fileName.endswith('xls') or fileName.endswith('xlsx')):
            count += 1
        else:
             print('WRONG')
        print(fileName,count)

In [ ]:
fromMapToOpener(filesMap)

In [26]:
testP = "files/Keppel/sep.pdf"
testN = "files/Keppel/2016_07_06_Singapore_Final_Country_Report.pdf"

In [27]:
import text_summarization
import extract_text
import stopword
import lemmatizer
import sys

In [28]:
text = extract_text.apply(testP)

In [29]:
summ = text_summarization.apply(text)
print(summ)

[' Keppelite I September 2008Keppel grows\nin Guangdong  4New tenants atMBFC 11Sporting success  31Special Focus: Keppelcontinues safety journeySpecial Focus: Keppelcontinues safety journeyKeppel Group NewsletterSEPTEMBER 2008www.kepcorp.com/ekeppelite\neppel\nKeppeliteMICA(P) NO.', '047/11/2007Special supplement in conjunction with Keppel CorporationÕs 40th anniversary with this issue\n Keppelite I September 2008Published for people in the Keppel Group by the Group Corporate Communications Division, Keppel Corporation Limited, 1 HarbourF\nront Ave, #18-01 Keppel Bay Tower,\nSingapore 098632.', 'Printed by Image Printers Pte Ltd, Blk 1002, Redhill Industrial Estate, Jalan Bukit Merah, #03-12, Singapore 159456.Editorial AdvisorWang Look Fung\nEditorMak Chui LingEditorial CommitteeAng Lai Lee, Andrew Ong, Catherine Tan, Chen Xi, Cheryl Tang, Chua Ying Ying, Chua Yong Hwee, Diana Chan, Donald Sng, Eva Ho, G\neraldine Lai, Grace Chia,Ho Ghim Khoon, Ivana Chua, Irene Lee, Jacquelyn Wong, Je

In [34]:
for i in summ:
    sumStop = stopword.removeStop(str(i))
    print(lemmatizer.lemmize(textPOS(sumStop)))

[('Keppelite', 'N'), ('I', 'P'), ('September', 'N'), ('2008Keppel', 'C'), ('grows', 'N'), ('Guangdong', 'N'), ('4New', 'C'), ('tenant', 'N'), ('atMBFC', 'J'), ('11Sporting', 'C'), ('success', 'N'), ('31Special', 'C'), ('Focus', 'N'), (':', ':'), ('Keppelcontinues', 'N'), ('safety', 'N'), ('journeySpecial', 'J'), ('Focus', 'N'), (':', ':'), ('Keppelcontinues', 'N'), ('safety', 'N'), ('journeyKeppel', 'N'), ('Group', 'N'), ('NewsletterSEPTEMBER', 'N'), ('2008www.kepcorp.com/ekeppelite', 'C'), ('eppel', 'N'), ('KeppeliteMICA', 'N'), ('(', '('), ('P', 'N'), (')', ')'), ('NO', 'N'), ('.', '.')]
[('047/11/2007Special', 'C'), ('supplement', 'J'), ('conjunction', 'N'), ('Keppel', 'N'), ('CorporationÕs', 'N'), ('40th', 'C'), ('anniversary', 'J'), ('issue', 'N'), ('Keppelite', 'N'), ('I', 'P'), ('September', 'N'), ('2008Published', 'C'), ('people', 'N'), ('Keppel', 'N'), ('Group', 'N'), ('Group', 'N'), ('Corporate', 'N'), ('Communications', 'N'), ('Division', 'N'), (',', ','), ('Keppel', 'N'), (

[('The', 'D'), ('vessel', 'N'), ('charter', 'V'), ('Petroleo', 'N'), ('Brasileiro', 'N'), ('SA', 'N'), ('(', '('), ('Petrobras', 'N'), (')', ')'), (',', ','), ('national', 'J'), ('oil', 'N'), ('company', 'N'), ('Brazil.and', 'N'), ('vigilance', 'N'), ('keep', 'N'), ('safetyas', 'N'), ('utmost', 'J'), ('priority', 'N'), (',', ','), ('even', 'R'), ('yard', 'N'), ('work', 'V'), ('optimum', 'J'), ('level.ÓProsafe', 'J'), (',', ','), ('one', 'C'), ('worldÕs', 'N'), ('lead', 'V'), ('owner', 'N'), ('operator', 'N'), ('FPSO', 'N'), ('FSO', 'N'), ('facility', 'N'), ('last', 'J'), ('10', 'C'), ('year', 'N'), ('entrust', 'V'), ('Keppel', 'N'), ('Shipyard', 'N'), ('exclusively', 'R'), ('carry', 'V'), ('conversion', 'N'), ('FPSO', 'N'), ('FSO', 'N'), ('system', 'N'), ('.', '.')]
[('keppeliteAt', 'N'), ('ceremony', 'N'), (',', ','), ('Mr', 'N'), ('Tong', 'N'), ('Chong', 'N'), ('Heong', 'N'), (',', ','), ('MD', 'N'), ('&', 'C'), ('COO', 'N'), ('Keppel', 'N'), ('Offshore', 'N'), ('&', 'C'), ('Marine',

[('keppeliteMr', 'N'), ('Linson', 'N'), ('Lim', 'N'), (',', ','), ('Chief', 'N'), ('Representative', 'N'), ('Keppel', 'N'), ('Land', 'N'), ('Vietnam', 'N'), ('(', '('), ('extreme', 'J'), ('left', 'N'), (')', ')'), (',', ','), ('present', 'J'), ('ceremony', 'N'), ('receive', 'V'), ('Medal', 'N'), ('Labour', 'N'), ('Award', 'N'), ('Vice-Chairman', 'N'), ('Nguyen', 'N'), ('Trung', 'N'), ('Tin', 'N'), ('(', '('), ('extreme', 'J'), ('right', 'N'), (')', ')'), ('HCMC', 'N'), ('PeopleÕs', 'N'), ('Committee', 'N'), ('22Special', 'C'), ('FocusKeppelite', 'N'), ('I', 'P'), ('September', 'N'), ('2008Safety', 'C'), ('marchKeppelites', 'N'), ('rally', 'R'), ('round', 'V'), ('ÒSafety', 'N'), ('Starts', 'V'), ('With', 'I'), ('MeÓ', 'N'), ('focus', 'N'), ('Keppel', 'N'), ('Group', 'N'), ('SafetyConvention', 'N'), ('2008.Safety', 'C'), ('expert', 'N'), ('Mr', 'N'), ('Roger', 'N'), ('Moore', 'N'), ('emphasise', 'V'), ('everyone', 'N'), ('must', 'M'), ('take', 'V'), ('responsibility', 'N'), ('safety', 'N

[('keppeliteBossa', 'N'), ('Nova', 'N'), ('legends', 'V'), ('(', '('), ('L-R', 'N'), (')', ')'), ('Joao', 'N'), ('Donato', 'N'), ('(', '('), ('piano', 'N'), (')', ')'), (',', ','), ('Wanda', 'N'), ('Sa', 'N'), (',', ','), ('Roberto', 'N'), ('Menescal', 'N'), (',', ','), ('Marcos', 'N'), ('Valle', 'N'), ('Patricia', 'N'), ('Alvi', 'N'), ('captivate', 'V'), ('aud', 'J'), ('ience', 'N'), ('flawless', 'N'), ('performance', 'N'), ('charm', 'V'), ('personalitiesAgility', 'N'), ('speed', 'N'), ('essential', 'J'), ('perform', 'V'), ('gravity-defying', 'J'), ('capoeira', 'N'), ('movesPhoto', 'N'), (':', ':'), ('John', 'N'), ('Yuen', 'N'), ('Team', 'N'), ('31Keppelite', 'C'), ('I', 'P'), ('September', 'N'), ('2008Having', 'C'), ('good', 'J'), ('run', 'N'), ('ofsporting', 'V'), ('activity', 'N'), ('start', 'V'), ('July', 'N'), (',', ','), ('Inter-Strategic', 'N'), ('Business', 'N'), ('Unit', 'N'), ('Games', 'N'), ('2008', 'C'), ('(', '('), ('iSBUG', 'N'), (')', ')'), ('draw', 'V'), ('close', 'J')

[('The', 'D'), ('segment', 'N'), ('Keppel', 'N'), ('FMO', 'N'), ('air', 'V'), ('18', 'C'), ('September', 'N'), ('2008', 'C'), ('at9.30', 'N'), ('pm', 'N'), ('CNA.In', 'N'), ('segment', 'N'), (',', ','), ('KeppelFMO', 'N'), ('employee', 'N'), ('film', 'V'), ('help', 'V'), ('client', 'N'), ('various', 'J'), ('facility', 'N'), ('management', 'N'), ('initiative', 'N'), (',', ','), ('include', 'V'), ('attain', 'V'), ('prestigious', 'J'), ('JCI', 'N'), ('accreditation', 'N'), ('.', '.')]
[('Together', 'R'), ('Alexandra', 'N'), ('Hospital', 'N'), (',', ','), ('SGP', 'N'), ('International', 'N'), ('Management', 'N'), ('CPGconsultants', 'N'), (',', ','), ('Keppel', 'N'), ('FMOis', 'N'), ('part', 'N'), ('Singapore', 'N'), ('Hospital', 'N'), ('Management', 'N'), ('Consortium', 'N'), ('(', '('), ('SHMC', 'N'), (')', ')'), ('.', '.')]
[('keppeliteDuring', 'V'), ('monthly', 'J'), ('luncheon', 'N'), ('talk', 'N'), ('organise', 'V'), ('Keppel', 'N'), ('LandÕs', 'N'), ('Welfare', 'N'), ('Committee', 'N

In [31]:
final = "" 
for i in summ:
    final += i+"\n"
test = (removeStop(final))
print(test)
test2 = textPOS(test)
print(t)

['Keppelite', 'September', '2008Keppel', 'grows', 'Guangdong', '4New', 'tenants', 'atMBFC', '11Sporting', 'success', '31Special', 'Focus', ':', 'Keppelcontinues', 'safety', 'journeySpecial', 'Focus', ':', 'Keppelcontinues', 'safety', 'journeyKeppel', 'Group', 'NewsletterSEPTEMBER', '2008www.kepcorp.com/ekeppelite', 'eppel', 'KeppeliteMICA', '(', 'P', ')', 'NO', '.', '047/11/2007Special', 'supplement', 'conjunction', 'Keppel', 'CorporationÕs', '40th', 'anniversary', 'issue', 'Keppelite', 'September', '2008Published', 'people', 'Keppel', 'Group', 'Group', 'Corporate', 'Communications', 'Division', ',', 'Keppel', 'Corporation', 'Limited', ',', '1', 'HarbourF', 'ront', 'Ave', ',', '#', '18-01', 'Keppel', 'Bay', 'Tower', ',', 'Singapore', '098632', '.', 'Printed', 'Image', 'Printers', 'Pte', 'Ltd', ',', 'Blk', '1002', ',', 'Redhill', 'Industrial', 'Estate', ',', 'Jalan', 'Bukit', 'Merah', ',', '#', '03-12', ',', 'Singapore', '159456.Editorial', 'AdvisorWang', 'Look', 'Fung', 'EditorMak', 'C

NameError: name 't' is not defined

# Test a classifier

In [36]:
from nltk.corpus import movie_reviews
from random import shuffle

In [57]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
for fileid in movie_reviews.fileids(category)]

In [107]:
shuffle(documents)
print(len(documents))

2000


In [59]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_fetures = list(all_words.keys())[:2000]

In [65]:
def docFeat(doc):
    document_words = set(doc)
    features = {}
    for word in word_fetures:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [66]:
featuresets = [(docFeat(d), c) for (d, c) in documents]

In [67]:
train_set,test_set = featuresets[100:], featuresets[:100]

In [117]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [118]:
#print(nltk.classify.accuracy(classifier,test_set))
classifier.show_most_informative_features()

Most Informative Features
       contains(miscast) = True              neg : pos    =      8.3 : 1.0
 contains(unimaginative) = True              neg : pos    =      7.7 : 1.0
        contains(suvari) = True              neg : pos    =      7.1 : 1.0
    contains(schumacher) = True              neg : pos    =      7.1 : 1.0
          contains(mena) = True              neg : pos    =      7.1 : 1.0
     contains(atrocious) = True              neg : pos    =      7.1 : 1.0
        contains(turkey) = True              neg : pos    =      6.8 : 1.0
        contains(wasted) = True              neg : pos    =      6.8 : 1.0
       contains(singers) = True              pos : neg    =      6.3 : 1.0
           contains(ugh) = True              neg : pos    =      5.8 : 1.0


In [121]:
print(classifier.classify(docFeat("turkey")))

neg


In [128]:
summ[25].replace("\n","").split()

['The',
 'plant',
 'is',
 'also',
 'oneof',
 'the',
 'first',
 'assets',
 'to',
 'beconsidered',
 'for',
 'injection',
 'intothe',
 'worldÕs',
 'first',
 'greenbusiness',
 'trust',
 'that',
 'will',
 'beset',
 'up',
 'and',
 'listed',
 'in',
 'Singaporeby',
 'Keppel',
 'IntegratedEngineering.']